<a href="https://colab.research.google.com/github/Vamsi-Malineni/Research-work/blob/master/pinn_trial_tf2_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import scipy.io
import numpy as np
import tensorflow as tf
import time
from datetime import datetime


In [15]:
def prepdata(N_train):
    #data = scipy.io.loadmat('Downloads/cylinder_nektar_wake.mat')
    data=scipy.io.loadmat('/content/drive/MyDrive/cylinder_nektar_wake.mat')
    U_star = data['U_star'] # N x 2 x T
    t_star = data['t'] # T x 1
    X_star = data['X_star'] # N x 2
    
    N = X_star.shape[0]
    T = t_star.shape[0]
    
    # Rearrange Data 
    XX = np.tile(X_star[:,0:1], (1,T)) # N x T
    YY = np.tile(X_star[:,1:2], (1,T)) # N x T
    TT = np.tile(t_star, (1,N)).T # N x T
    
    UU = U_star[:,0,:] # N x T
    VV = U_star[:,1,:] # N x T
    
    x = XX.flatten()[:,None] # NT x 1
    y = YY.flatten()[:,None] # NT x 1
    t = TT.flatten()[:,None] # NT x 1
    
    u = UU.flatten()[:,None] # NT x 1
    v = VV.flatten()[:,None] # NT x 1
    
    # Training Data    
    idx = np.random.choice(N*T, N_train, replace=False)
    x_train = x[idx,:]
    y_train = y[idx,:]
    t_train = t[idx,:]
    u_train = u[idx,:]
    v_train = v[idx,:]
    
    lb=tf.math.reduce_min(tf.concat([x_train,y_train,t_train],1),keepdims=True,axis=0) 
    ub=tf.math.reduce_max(tf.concat([x_train,y_train,t_train],1),keepdims=True,axis=0)
    lb=tf.cast(lb,dtype=tf.float32)
    ub=tf.cast(ub,dtype=tf.float32)

    return x_train,y_train,t_train,u_train,v_train,lb,ub


In [16]:
class Logger(object):
  def __init__(self, frequency=10):
    print("TensorFlow version: {}".format(tf.__version__))
    print("Eager execution: {}".format(tf.executing_eagerly()))
    print("GPU-accerelated: {}".format(tf.test.is_gpu_available()))

    self.start_time = time.time()
    self.frequency = frequency

  def __get_elapsed(self):
    return datetime.fromtimestamp(time.time() - self.start_time).strftime("%M:%S")

  def __get_error_u(self):
    return self.error_fn()

  def set_error_fn(self, error_fn):
    self.error_fn = error_fn
  
  def log_train_start(self, model):
    print("\nTraining started")
    print("================")
    self.model = model
    print(self.model.summary())

  def log_train_epoch(self, epoch, loss, custom="", is_iter=False):
    if epoch % self.frequency == 0:
      print(f"{'nt_epoch' if is_iter else 'tf_epoch'} = {epoch:6d}  elapsed = {self.__get_elapsed()}  loss = {loss:.4e}  error = {self.__get_error_u():.4e}  " + custom)

  def log_train_opt(self, name):
    # print(f"tf_epoch =      0  elapsed = 00:00  loss = 2.7391e-01  error = 9.0843e-01")
    print(f"—— Starting {name} optimization ——")

  def log_train_end(self, epoch, custom=""):
    print("==================")
    print(f"Training finished (epoch {epoch}): duration = {self.__get_elapsed()}  error = {self.__get_error_u():.4e}  " + custom)

In [17]:
#@title LBFGS
# Time tracking functions
global_time_list = []
global_last_time = 0
def reset_time():
  global global_time_list, global_last_time
  global_time_list = []
  global_last_time = time.perf_counter()
  
def record_time():
  global global_last_time, global_time_list
  new_time = time.perf_counter()
  global_time_list.append(new_time - global_last_time)
  global_last_time = time.perf_counter()
  #print("step: %.2f"%(global_time_list[-1]*1000))

def last_time():
  """Returns last interval records in millis."""
  global global_last_time, global_time_list
  if global_time_list:
    return 1000 * global_time_list[-1]
  else:
    return 0

def dot(a, b):
  """Dot product function since TensorFlow doesn't have one."""
  return tf.reduce_sum(a*b)

def verbose_func(s):
  print(s)

final_loss = None
times = []
def lbfgs(opfunc, x, config, state, do_verbose, log_fn):
  """port of lbfgs.lua, using TensorFlow eager mode.
  """

  if config.maxIter == 0:
    return

  global final_loss, times
  
  maxIter = config.maxIter
  maxEval = config.maxEval or maxIter*1.25
  tolFun = config.tolFun or 1e-5
  tolX = config.tolX or 1e-19
  nCorrection = config.nCorrection or 100
  lineSearch = config.lineSearch
  lineSearchOpts = config.lineSearchOptions
  learningRate = config.learningRate or 1
  isverbose = config.verbose or False

  # verbose function
  if isverbose:
    verbose = verbose_func
  else:
    verbose = lambda x: None

    # evaluate initial f(x) and df/dx
  f, g = opfunc(x)

  f_hist = [f]
  currentFuncEval = 1
  state.funcEval = state.funcEval + 1
  p = g.shape[0]

  # check optimality of initial point
  tmp1 = tf.abs(g)
  if tf.reduce_sum(tmp1) <= tolFun:
    verbose("optimality condition below tolFun")
    return x, f_hist

  # optimize for a max of maxIter iterations
  nIter = 0
  times = []
  while nIter < maxIter:
    start_time = time.time()
    
    # keep track of nb of iterations
    nIter = nIter + 1
    state.nIter = state.nIter + 1

    ############################################################
    ## compute gradient descent direction
    ############################################################
    if state.nIter == 1:
      d = -g
      old_dirs = []
      old_stps = []
      Hdiag = 1
    else:
      # do lbfgs update (update memory)
      y = g - g_old
      s = d*t
      ys = dot(y, s)
      
      if ys > 1e-10:
        # updating memory
        if len(old_dirs) == nCorrection:
          # shift history by one (limited-memory)
          del old_dirs[0]
          del old_stps[0]

        # store new direction/step
        old_dirs.append(s)
        old_stps.append(y)

        # update scale of initial Hessian approximation
        Hdiag = ys/dot(y, y)

      # compute the approximate (L-BFGS) inverse Hessian 
      # multiplied by the gradient
      k = len(old_dirs)

      # need to be accessed element-by-element, so don't re-type tensor:
      ro = [0]*nCorrection
      for i in range(k):
        ro[i] = 1/dot(old_stps[i], old_dirs[i])
        

      # iteration in L-BFGS loop collapsed to use just one buffer
      # need to be accessed element-by-element, so don't re-type tensor:
      al = [0]*nCorrection

      q = -g
      for i in range(k-1, -1, -1):
        al[i] = dot(old_dirs[i], q) * ro[i]
        q = q - al[i]*old_stps[i]

      # multiply by initial Hessian
      r = q*Hdiag
      for i in range(k):
        be_i = dot(old_stps[i], r) * ro[i]
        r += (al[i]-be_i)*old_dirs[i]
        
      d = r
      # final direction is in r/d (same object)

    g_old = g
    f_old = f
    
    ############################################################
    ## compute step length
    ############################################################
    # directional derivative
    gtd = dot(g, d)

    # check that progress can be made along that direction
    if gtd > -tolX:
      verbose("Can not make progress along direction.")
      break

    # reset initial guess for step size
    if state.nIter == 1:
      tmp1 = tf.abs(g)
      t = min(1, 1/tf.reduce_sum(tmp1))
    else:
      t = learningRate


    # optional line search: user function
    lsFuncEval = 0
    if lineSearch and isinstance(lineSearch) == types.FunctionType:
      # perform line search, using user function
      f,g,x,t,lsFuncEval = lineSearch(opfunc,x,t,d,f,g,gtd,lineSearchOpts)
      f_hist.append(f)
    else:
      # no line search, simply move with fixed-step
      x += t*d
      
      if nIter != maxIter:
        # re-evaluate function only if not in last iteration
        # the reason we do this: in a stochastic setting,
        # no use to re-evaluate that function here
        f, g = opfunc(x)
        lsFuncEval = 1
        f_hist.append(f)


    # update func eval
    currentFuncEval = currentFuncEval + lsFuncEval
    state.funcEval = state.funcEval + lsFuncEval

    ############################################################
    ## check conditions
    ############################################################
    if nIter == maxIter:
      break

    if currentFuncEval >= maxEval:
      # max nb of function evals
      verbose('max nb of function evals')
      break

    tmp1 = tf.abs(g)
    if tf.reduce_sum(tmp1) <=tolFun:
      # check optimality
      verbose('optimality condition below tolFun')
      break
    
    tmp1 = tf.abs(d*t)
    if tf.reduce_sum(tmp1) <= tolX:
      # step size below tolX
      verbose('step size below tolX')
      break

    if tf.abs(f-f_old) < tolX:
      # function value changing less than tolX
      verbose('function value changing less than tolX'+str(tf.abs(f-f_old)))
      break

    if do_verbose:
      log_fn(nIter, f.numpy(), True)
      #print("Step %3d loss %6.5f msec %6.3f"%(nIter, f.numpy(), last_time()))
      record_time()
      times.append(last_time())

    if nIter == maxIter - 1:
      final_loss = f.numpy()


  # save state
  state.old_dirs = old_dirs
  state.old_stps = old_stps
  state.Hdiag = Hdiag
  state.g_old = g_old
  state.f_old = f_old
  state.t = t
  state.d = d

  return x, f_hist, currentFuncEval

# dummy/Struct gives Lua-like struct object with 0 defaults
class dummy(object):
  pass

class Struct(dummy):
  def __getattribute__(self, key):
    if key == '__dict__':
      return super(dummy, self).__getattribute__('__dict__')
    return self.__dict__.get(key, 0)

In [18]:
#@title HYPER PARAMETERS

# Data size on the solution u
N_u = 2000
# Setting up the TF SGD-based optimizer (set tf_epochs=0 to cancel it)
tf_epochs = 100
tf_optimizer = tf.keras.optimizers.Adam(
  learning_rate=0.001)
# Setting up the quasi-newton LBGFS optimizer (set nt_epochs=0 to cancel it)
nt_epochs = 1000
nt_config = Struct()
nt_config.learningRate = 0.8
nt_config.maxIter = nt_epochs
nt_config.nCorrection = 50
nt_config.tolFun = 1.0 * np.finfo(float).eps


In [19]:

class PhysicsInformedNN(object):
  def __init__(self, layers, optimizer, logger, ub, lb):
    # Descriptive Keras model [2, 20, …, 20, 1]
    self.u_model = tf.keras.Sequential()
    self.u_model.add(tf.keras.layers.InputLayer(input_shape=(layers[0],)))
    self.u_model.add(tf.keras.layers.Lambda(
      lambda X: 2.0*(X - lb)/(ub - lb) - 1.0))
    for width in layers[1:]:
        self.u_model.add(tf.keras.layers.Dense(
          width, activation=tf.nn.tanh,
          kernel_initializer='glorot_normal'))

    # Computing the sizes of weights/biases for future decomposition
    self.sizes_w = []
    self.sizes_b = []
    for i, width in enumerate(layers):
      if i != 1:
        self.sizes_w.append(int(width * layers[1]))
        self.sizes_b.append(int(width if i != 0 else layers[1]))

    self.dtype = tf.float32

    # Defining the two additional trainable variables for identification
    self.lambda_1 = tf.Variable([0.0], dtype=self.dtype)
    self.lambda_2 = tf.Variable([-6.0], dtype=self.dtype)
    
    self.optimizer = optimizer
    self.logger = logger

  # The actual PINN
  def __f_model(self, xtrain,ytrain,ttrain):
    l1, l2 = self.get_params()
    # Separating the collocation coordinates
    x_f=tf.convert_to_tensor(xtrain,dtype=self.dtype)
    y_f=tf.convert_to_tensor(ytrain,dtype=self.dtype)
    t_f=tf.convert_to_tensor(ttrain,dtype=self.dtype)
    
    with tf.GradientTape(persistent=True) as g:
      g.watch(x_f)
      g.watch(y_f)
      g.watch(t_f)
      # Stacking the input variables and sending them to model
      xf=tf.stack([x_f[:,0],y_f[:,0],t_f[:,0]],axis=1)

      psi_and_p=self.u_model(xf)

      psi=psi_and_p[:,0:1]
      p=psi_and_p[:,1:2]

      u= g.gradient(psi,y_f)
      v=-g.gradient(psi,x_f)

      ut=g.gradient(u,t_f)
      ux=g.gradient(u,x_f)
      uy=g.gradient(u,y_f)
      
      vt=g.gradient(v,t_f)
      vx=g.gradient(v,x_f)
      vy=g.gradient(v,y_f)

      px=g.gradient(p,x_f)
      py=g.gradient(p,y_f)
    
    uxx=g.gradient(ux,x_f)
    uyy=g.gradient(uy,y_f)
    vxx=g.gradient(vx,x_f)
    vyy=g.gradient(vy,y_f)

    del g
    f = ut+l1*(u*ux+v*uy)+px-l2*(uxx+uyy) 
    g = vt+l1*(u*vx+v*vy)+py-l2*(vxx+vyy)
    
    return u,v,p,f,g
  # Defining custom loss
  def __loss(self, x,y,t,u,v):
    u_pred,v_pred,p_pred,f_pred,g_pred=self.__f_model(x,y,t)
      
    loss_eq=tf.reduce_mean(tf.square(u-u_pred))+ \
            tf.reduce_mean(tf.square(v-v_pred))+ \
            tf.reduce_mean(tf.square(f_pred))+ \
            tf.reduce_mean(tf.square(g_pred))
    return loss_eq

  def __grad(self, x,y,t,u,v):
    with tf.GradientTape() as tape:
      loss_value = self.__loss(x,y,t,u,v)
    return loss_value, tape.gradient(loss_value, self.__wrap_training_variables())

  def __wrap_training_variables(self):
    var = self.u_model.trainable_variables
    var.extend([self.lambda_1, self.lambda_2])
    return var

  def get_weights(self):
      w = []
      for layer in self.u_model.layers[1:]:
        weights_biases = layer.get_weights()
        weights = weights_biases[0].flatten()
        biases = weights_biases[1]
        w.extend(weights)
        w.extend(biases)
      w.extend(self.lambda_1.numpy())
      w.extend(self.lambda_2.numpy())
      return tf.convert_to_tensor(w, dtype=self.dtype)

  def set_weights(self, w):
    for i, layer in enumerate(self.u_model.layers[1:]):
      start_weights = sum(self.sizes_w[:i]) + sum(self.sizes_b[:i])
      end_weights = sum(self.sizes_w[:i+1]) + sum(self.sizes_b[:i])
      weights = w[start_weights:end_weights]
      w_div = int(self.sizes_w[i] / self.sizes_b[i])
      weights = tf.reshape(weights, [w_div, self.sizes_b[i]])
      biases = w[end_weights:end_weights + self.sizes_b[i]]
      weights_biases = [weights, biases]
      layer.set_weights(weights_biases)
    self.lambda_1.assign([w[-2]])
    self.lambda_2.assign([w[-1]])

  def get_params(self, numpy=False):
    l1 = self.lambda_1
    l2 = tf.exp(self.lambda_2)
    if numpy:
      return l1.numpy()[0], l2.numpy()[0]
    return l1, l2

  def summary(self):
    return self.u_model.summary()

  # The training function
  def fit(self,x,y,t,u,v, tf_epochs, nt_config):
    self.logger.log_train_start(self)

    # Creating the tensors
    x=tf.convert_to_tensor(x,dtype=self.dtype)
    y=tf.convert_to_tensor(y,dtype=self.dtype)
    t=tf.convert_to_tensor(t,dtype=self.dtype)
    u=tf.convert_to_tensor(u,dtype=self.dtype)
    v=tf.convert_to_tensor(v,dtype=self.dtype)
      

    def log_train_epoch(epoch, loss, is_iter):
      l1, l2 = self.get_params(numpy=True)
      custom = f"l1 = {l1:5f}  l2 = {l2:8f}"
      self.logger.log_train_epoch(epoch, loss, custom, is_iter)

    self.logger.log_train_opt("Adam")
    for epoch in range(tf_epochs):
      # Optimization step
      loss_value, grads = self.__grad(x,y,t,u,v)
      self.optimizer.apply_gradients(
        zip(grads, self.__wrap_training_variables()))
      log_train_epoch(epoch, loss_value, False)

    self.logger.log_train_opt("LBFGS")
   
    def loss_and_flat_grad(w):
      with tf.GradientTape() as tape:
        self.set_weights(w)
        tape.watch(self.lambda_1)
        tape.watch(self.lambda_2)
        loss_value = self.__loss(x,y,t,u,v)
      grad = tape.gradient(loss_value, self.__wrap_training_variables())
      grad_flat = []
      for g in grad:
        grad_flat.append(tf.reshape(g, [-1]))
      grad_flat =  tf.concat(grad_flat, 0)
      return loss_value, grad_flat
   
    lbfgs(loss_and_flat_grad,
      self.get_weights(),
      nt_config, Struct(), True, log_train_epoch)
    
    l1, l2 = self.get_params(numpy=True)
    self.logger.log_train_end(tf_epochs, f"l1 = {l1:5f}  l2 = {l2:8f}")

  def predict(self, X_star):
    u_star = self.u_model(X_star)
    f_star = self.__f_model(X_star)
    return u_star.numpy(), f_star.numpy()

In [20]:
x,y,t,u,v,lb,ub=prepdata(N_u)
layers=[3,20,20,20,20,20,20,20,20,2]
lambdas_star=[1,0.01]

logger = Logger(frequency=10)
pinn = PhysicsInformedNN(layers, tf_optimizer, logger, ub, lb)
def error():
  l1, l2 = pinn.get_params(numpy=True)
  l1_star, l2_star = lambdas_star
  error_lambda_1 = np.abs(l1 - l1_star) / l1_star
  error_lambda_2 = np.abs(l2 - l2_star) / l2_star
  return (error_lambda_1 + error_lambda_2) / 2
logger.set_error_fn(error)
pinn.fit(x,y,t,u,v,tf_epochs, nt_config)
lambda_1_pred, lambda_2_pred = pinn.get_params(numpy=True)
print("l1: ", lambda_1_pred)
print("l2: ", lambda_2_pred)

TensorFlow version: 2.8.0
Eager execution: True
GPU-accerelated: False

Training started
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_3 (Lambda)           (None, 3)                 0         
                                                                 
 dense_27 (Dense)            (None, 20)                80        
                                                                 
 dense_28 (Dense)            (None, 20)                420       
                                                                 
 dense_29 (Dense)            (None, 20)                420       
                                                                 
 dense_30 (Dense)            (None, 20)                420       
                                                                 
 dense_31 (Dense)            (None, 20)                420       
                               

In [21]:
print(layers)

[3, 20, 20, 20, 20, 20, 20, 20, 20, 2]
